# Gráficos de Cleveland: CT Izquierda vs Derecha

Este notebook genera **gráficos de Cleveland** para visualizar el Cambio de Tiempo (CT) comparando candidatos de **Izquierda** vs **Derecha**.

## Concepto:

Para cada ítem mostramos:
- **Punto azul (●)**: Tiempo promedio en candidatos de **Izquierda**
- **Punto rojo (●)**: Tiempo promedio en candidatos de **Derecha**
- **Línea conectando**: Muestra la asimetría ideológica
- **Color de línea**:
  - Verde: Mayor tiempo en Izquierda (>0.5 seg)
  - Gris: Tiempos similares (±0.5 seg)
  - Naranja: Mayor tiempo en Derecha (<-0.5 seg)

## Interpretación:

- **Valores más altos**: Mayor tiempo de respuesta
- **Línea hacia la derecha**: Más tiempo en candidatos de Izquierda
- **Línea hacia la izquierda**: Más tiempo en candidatos de Derecha
- **Hipótesis**: Diferencias pueden indicar procesamiento más deliberado según afinidad ideológica

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Librerías cargadas exitosamente')

## 1. Cargar Datos

In [ ]:
# Ruta al archivo
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Procesados')
Archivo_Elecciones = os.path.join(Ruta_Base, 'df_Elecciones.xlsx')

# Cargar datos
df_Elecciones = pd.read_excel(Archivo_Elecciones)

print(f'✓ Datos cargados:')
print(f'  - {len(df_Elecciones)} registros')
print(f'  - {len(df_Elecciones.columns)} columnas')

# Verificar columnas CT
columnas_ct_izq = [col for col in df_Elecciones.columns if col.startswith('CT_Item_') and col.endswith('_Izq')]
columnas_ct_der = [col for col in df_Elecciones.columns if col.startswith('CT_Item_') and col.endswith('_Der')]

print(f'\n  - Variables CT_Izq: {len(columnas_ct_izq)}')
print(f'  - Variables CT_Der: {len(columnas_ct_der)}')

## 2. Preparar Datos para Gráfico

In [ ]:
# Definir ítems
Items_Progresistas = [5, 6, 9, 11, 16, 20, 24, 25, 27, 28]
Items_Conservadores = [3, 4, 7, 8, 10, 19, 22, 23, 29, 30]
Todos_Items = sorted(Items_Progresistas + Items_Conservadores)

print(f'Total de ítems: {len(Todos_Items)}')
print(f'Ítems: {Todos_Items}')

# Calcular promedios por ítem
datos_grafico = []

for item in Todos_Items:
    var_izq = f'CT_Item_{item}_Izq'
    var_der = f'CT_Item_{item}_Der'
    
    if var_izq in df_Elecciones.columns and var_der in df_Elecciones.columns:
        # Calcular promedios (ignorando NaN)
        media_izq = df_Elecciones[var_izq].mean()
        media_der = df_Elecciones[var_der].mean()
        
        # Calcular diferencia (Izq - Der)
        diferencia = media_izq - media_der
        
        # Determinar tipo de ítem
        tipo = 'Progresista' if item in Items_Progresistas else 'Conservador'
        
        datos_grafico.append({
            'Item': item,
            'Tipo': tipo,
            'CT_Izquierda': media_izq,
            'CT_Derecha': media_der,
            'Diferencia': diferencia
        })

# Crear DataFrame
df_grafico = pd.DataFrame(datos_grafico)

print(f'\n✓ Datos preparados para {len(df_grafico)} ítems')
print(f'\nPrimeras filas:')
print(df_grafico.head(10))

## 3. Función para Crear Gráfico de Cleveland

In [ ]:
def Crear_Grafico_Cleveland_CT(df, titulo, nombre_archivo=None, carpeta_destino='Graficos_Cleveland'):
    """
    Crea un gráfico de Cleveland mostrando CT Izquierda vs Derecha.
    
    Parámetros:
    -----------
    df : DataFrame
        DataFrame con columnas: Item, CT_Izquierda, CT_Derecha, Diferencia
    titulo : str
        Título del gráfico
    nombre_archivo : str, optional
        Nombre del archivo (sin extensión)
    carpeta_destino : str
        Carpeta donde guardar
    """
    
    # Crear carpeta si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)
    
    # Ordenar por diferencia (descendente)
    df_sorted = df.sort_values('Diferencia', ascending=True).reset_index(drop=True)
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Parámetros visuales
    y_positions = np.arange(len(df_sorted))
    
    # Dibujar líneas conectando los puntos
    for idx, row in df_sorted.iterrows():
        izq_val = row['CT_Izquierda']
        der_val = row['CT_Derecha']
        diferencia = row['Diferencia']
        
        # Determinar color de línea según diferencia
        if diferencia > 0.5:  # Más tiempo en Izquierda
            color_linea = '#3498db'  # Azul
            alpha = 0.6
        elif diferencia < -0.5:  # Más tiempo en Derecha
            color_linea = '#e74c3c'  # Rojo
            alpha = 0.6
        else:  # Diferencia pequeña
            color_linea = '#95a5a6'  # Gris
            alpha = 0.3
        
        # Dibujar línea
        ax.plot([der_val, izq_val], [idx, idx], 
                color=color_linea, linewidth=1.5, alpha=alpha, zorder=1)
    
    # Dibujar puntos de Izquierda
    ax.scatter(df_sorted['CT_Izquierda'], y_positions, 
               s=120, c='#3498db', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Izquierda', zorder=3, alpha=0.8)
    
    # Dibujar puntos de Derecha
    ax.scatter(df_sorted['CT_Derecha'], y_positions, 
               s=120, c='#e74c3c', marker='o', 
               edgecolors='white', linewidths=1.5,
               label='Derecha', zorder=3, alpha=0.8)
    
    # Configurar ejes
    ax.set_yticks(y_positions)
    ax.set_yticklabels([f'Ítem {int(item)}' for item in df_sorted['Item']], fontsize=10)
    
    ax.set_xlabel('Tiempo de Respuesta (segundos)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Ítem', fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    
    # Grid
    ax.grid(True, axis='x', alpha=0.3, linestyle=':', linewidth=0.5)
    ax.set_axisbelow(True)
    
    # Leyenda personalizada
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', 
               markersize=10, label='Izquierda', markeredgecolor='white', markeredgewidth=1),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='#e74c3c', 
               markersize=10, label='Derecha', markeredgecolor='white', markeredgewidth=1),
        Line2D([0], [0], color='#3498db', linewidth=2, label='Más tiempo Izq (>0.5s)'),
        Line2D([0], [0], color='#95a5a6', linewidth=2, label='Similar (±0.5s)'),
        Line2D([0], [0], color='#e74c3c', linewidth=2, label='Más tiempo Der (<-0.5s)')
    ]
    
    ax.legend(handles=legend_elements, loc='lower right', fontsize=10, 
              framealpha=0.95, edgecolor='gray')
    
    # Ajustar layout
    plt.tight_layout()
    
    # Guardar
    if nombre_archivo:
        ruta_completa = os.path.join(carpeta_destino, f'{nombre_archivo}.png')
        plt.savefig(ruta_completa, dpi=300, bbox_inches='tight', facecolor='white')
        print(f'✅ Gráfico guardado: {ruta_completa}')
    
    plt.show()
    
    return fig, ax

## 4. Gráfico Principal: Todos los Ítems

In [ ]:
# Crear gráfico principal
fig, ax = Crear_Grafico_Cleveland_CT(
    df_grafico,
    titulo='Tiempo de Respuesta:\nCandidatos de Izquierda vs Derecha',
    nombre_archivo='Cleveland_CT_Izquierda_vs_Derecha'
)

## 5. Análisis Estadístico de Diferencias

In [ ]:
print('='*70)
print('ANÁLISIS: CT IZQUIERDA VS DERECHA')
print('='*70)

print(f'\n📊 Estadísticas Generales:')
print(f'  Promedio CT_Izquierda: {df_grafico["CT_Izquierda"].mean():.3f} seg')
print(f'  Promedio CT_Derecha: {df_grafico["CT_Derecha"].mean():.3f} seg')
print(f'  Diferencia promedio (Izq - Der): {df_grafico["Diferencia"].mean():.3f} seg')

print(f'\n📈 Distribución de Diferencias:')
mayor_izq = len(df_grafico[df_grafico['Diferencia'] > 0.5])
similares = len(df_grafico[(df_grafico['Diferencia'] >= -0.5) & (df_grafico['Diferencia'] <= 0.5)])
mayor_der = len(df_grafico[df_grafico['Diferencia'] < -0.5])

print(f'  Mayor tiempo en Izquierda (>0.5s): {mayor_izq} ítems ({mayor_izq/len(df_grafico)*100:.1f}%)')
print(f'  Similar (±0.5s): {similares} ítems ({similares/len(df_grafico)*100:.1f}%)')
print(f'  Mayor tiempo en Derecha (<-0.5s): {mayor_der} ítems ({mayor_der/len(df_grafico)*100:.1f}%)')

print(f'\n🏆 Top 5 Ítems con MÁS tiempo en Izquierda:')
print('-'*70)
top_izq = df_grafico.nlargest(5, 'Diferencia')[['Item', 'Tipo', 'CT_Izquierda', 'CT_Derecha', 'Diferencia']]
print(top_izq.to_string(index=False))

print(f'\n🏆 Top 5 Ítems con MÁS tiempo en Derecha:')
print('-'*70)
top_der = df_grafico.nsmallest(5, 'Diferencia')[['Item', 'Tipo', 'CT_Izquierda', 'CT_Derecha', 'Diferencia']]
print(top_der.to_string(index=False))

print('\n' + '='*70)

## 6. Test Estadístico: ¿Son Diferentes?

In [ ]:
from scipy import stats

print('='*70)
print('TEST ESTADÍSTICO')
print('='*70)

# Test t pareado (porque son los mismos ítems en dos condiciones)
t_stat, p_value = stats.ttest_rel(df_grafico['CT_Izquierda'], df_grafico['CT_Derecha'])

print('\n📊 Test t Pareado:')
print('-'*70)
print(f'  H0: No hay diferencia entre CT_Izquierda y CT_Derecha')
print(f'  Ha: Hay diferencia significativa')
print(f'\n  Estadístico t: {t_stat:.4f}')
print(f'  Valor p: {p_value:.4f}')

if p_value < 0.05:
    print(f'\n  ✅ SIGNIFICATIVO (p < 0.05)')
    if df_grafico['Diferencia'].mean() > 0:
        print(f'     El tiempo es SIGNIFICATIVAMENTE MAYOR en candidatos de Izquierda')
    else:
        print(f'     El tiempo es SIGNIFICATIVAMENTE MAYOR en candidatos de Derecha')
else:
    print(f'\n  ❌ NO SIGNIFICATIVO (p ≥ 0.05)')
    print(f'     No hay diferencia significativa entre candidatos')

# Tamaño del efecto (d de Cohen)
diferencias = df_grafico['CT_Izquierda'] - df_grafico['CT_Derecha']
d_cohen = diferencias.mean() / diferencias.std()

print(f'\n  Tamaño del efecto (d de Cohen): {d_cohen:.4f}')
if abs(d_cohen) < 0.2:
    magnitud = 'PEQUEÑO'
elif abs(d_cohen) < 0.5:
    magnitud = 'MEDIANO'
else:
    magnitud = 'GRANDE'
print(f'  Magnitud: {magnitud}')

print('\n' + '='*70)

## 7. Análisis por Tipo de Ítem: ¿Interacción?

In [ ]:
print('='*70)
print('ANÁLISIS POR TIPO DE ÍTEM')
print('='*70)

# Agrupar por tipo
por_tipo = df_grafico.groupby('Tipo').agg({
    'CT_Izquierda': ['mean', 'std'],
    'CT_Derecha': ['mean', 'std'],
    'Diferencia': ['mean', 'std'],
    'Item': 'count'
}).round(3)

print('\n📊 Estadísticas por Tipo:')
print(por_tipo)

print('\n💡 Interpretación Teórica:')
df_prog = df_grafico[df_grafico['Tipo'] == 'Progresista']
df_cons = df_grafico[df_grafico['Tipo'] == 'Conservador']

dif_prog = df_prog['Diferencia'].mean()
dif_cons = df_cons['Diferencia'].mean()

print(f'\n  Ítems Progresistas:')
print(f'    - Diferencia promedio (Izq - Der): {dif_prog:.3f} seg')
if dif_prog > 0.5:
    print(f'    - ✅ ESPERADO: Candidatos de Izquierda tardan MÁS en ítems progresistas')
    print(f'    - (Mayor procesamiento de contenido afín)')
elif dif_prog < -0.5:
    print(f'    - ⚠️  INESPERADO: Candidatos de Derecha tardan MÁS en ítems progresistas')
else:
    print(f'    - ➖ Sin diferencia clara entre candidatos')

print(f'\n  Ítems Conservadores:')
print(f'    - Diferencia promedio (Izq - Der): {dif_cons:.3f} seg')
if dif_cons < -0.5:
    print(f'    - ✅ ESPERADO: Candidatos de Derecha tardan MÁS en ítems conservadores')
    print(f'    - (Mayor procesamiento de contenido afín)')
elif dif_cons > 0.5:
    print(f'    - ⚠️  INESPERADO: Candidatos de Izquierda tardan MÁS en ítems conservadores')
else:
    print(f'    - ➖ Sin diferencia clara entre candidatos')

print('\n🎯 Patrón de Interacción:')
if (dif_prog > 0 and dif_cons < 0) or (dif_prog < 0 and dif_cons > 0):
    print('  ✅ HAY INTERACCIÓN: El efecto de ideología cambia según tipo de ítem')
    print('     (Patrón esperado: mayor tiempo en contenido ideológicamente afín)')
else:
    print('  ➖ NO HAY INTERACCIÓN: El efecto es consistente entre tipos')

print('\n' + '='*70)

## 8. Comparación por Tipo de Ítem

In [ ]:
print('='*70)
print('COMPARACIÓN: PROGRESISTAS vs CONSERVADORES')
print('='*70)

# Test t para comparar diferencias entre tipos
t_stat_tipos, p_value_tipos = stats.ttest_ind(df_prog['Diferencia'], df_cons['Diferencia'])

print('\n📊 Test t: ¿Las diferencias (Izq-Der) varían por tipo de ítem?')
print('-'*70)
print(f'  H0: Las diferencias Izq-Der son iguales entre tipos de ítem')
print(f'  Ha: Las diferencias Izq-Der difieren según tipo de ítem')
print(f'\n  Estadístico t: {t_stat_tipos:.4f}')
print(f'  Valor p: {p_value_tipos:.4f}')

if p_value_tipos < 0.05:
    print(f'\n  ✅ SIGNIFICATIVO (p < 0.05)')
    print(f'     La asimetría Izq-Der DEPENDE del tipo de ítem')
else:
    print(f'\n  ❌ NO SIGNIFICATIVO (p ≥ 0.05)')
    print(f'     La asimetría Izq-Der NO depende del tipo de ítem')

print('\n' + '='*70)

## 9. Guardar Tabla Resumen

In [ ]:
# Crear carpeta de salida
Carpeta_Salida = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_Cleveland')
if not os.path.exists(Carpeta_Salida):
    os.makedirs(Carpeta_Salida)

# Guardar tabla de datos
Archivo_Salida = os.path.join(Carpeta_Salida, 'Resumen_CT_Izquierda_vs_Derecha.xlsx')
df_grafico_export = df_grafico.copy()
df_grafico_export.to_excel(Archivo_Salida, index=False)

print(f'✅ Tabla resumen guardada: {Archivo_Salida}')
print(f'   {len(df_grafico_export)} ítems × {len(df_grafico_export.columns)} columnas')

## 10. Resumen Final

In [ ]:
print('='*70)
print('RESUMEN: GRÁFICOS DE CLEVELAND - CT IZQUIERDA VS DERECHA')
print('='*70)

print('\n📊 Análisis completado:')
print(f'  - Ítems analizados: {len(df_grafico)}')
print(f'  - Gráficos generados: 1 (Cleveland plot)')
print(f'  - Comparación: Candidatos Izquierda vs Derecha')

print('\n📁 Archivos generados:')
print('  - Cleveland_CT_Izquierda_vs_Derecha.png')
print('  - Resumen_CT_Izquierda_vs_Derecha.xlsx')

print('\n🎯 Hallazgos clave:')
print(f'  - {mayor_izq} ítems: Más tiempo en Izquierda')
print(f'  - {mayor_der} ítems: Más tiempo en Derecha')
print(f'  - {similares} ítems: Tiempos similares')

print('\n💡 Interpretación:')
print('  - PROGRESISTAS: Diferencia promedio = {:.3f} seg'.format(dif_prog))
print('  - CONSERVADORES: Diferencia promedio = {:.3f} seg'.format(dif_cons))

print('\n' + '='*70)
print('✅ ANÁLISIS COMPLETADO')
print('='*70)